In [26]:
%pip install jovian --upgrade --quiet
%pip install requests --upgrade --quiet
%pip install beautifulsoup4 --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [27]:
import jovian
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PIL import Image
from pytesseract import pytesseract

In [28]:
# download the web page using requests
response = requests.get('https://hosocongty.vn/p/ha-noi')

#If response status is between 200-300, the download was successful
response

<Response [200]>

In [29]:
# capture the text content of the page from response
page_contents = response.text

In [30]:
# check the lenght of the page text
len(page_contents)

21771

In [31]:
# check a small sample of the page
page_contents[:500]

'<!DOCTYPE html>\n<html lang="vi">\n<head><meta http-equiv="content-type" content="text/html; charset=utf-8" /><base href="https://hosocongty.vn/"/>\n<meta name="robots" content="follow, all" />\n<meta name="keywords" content="Có 582,133 hồ sơ công ty tại Thành phố Hà Nội,Tra cứu mã số thuế, Tra cứu thông tin doanh nghiệp, Tra cứu doanh nghiêp, Tra cứu công ty, Hồ Sơ công ty"/>\n<meta name="description" content="Có 582,133 hồ sơ công ty tại Thành phố Hà Nội"/>\n<meta name="viewport" content="width=devi'

In [32]:
# Save the page as html to avoid scrapping multiple times while experimenting
with open('ho_so_cong_ty.html', 'w', encoding="utf-8") as file:
    file.write(page_contents)

In [33]:
# read it html page to scrape information
with open('ho_so_cong_ty.html', 'r', encoding="utf-8") as f:
    html_source = f.read()

In [34]:
#doublecheck the file before exploring the tags. It contrins the webpage text as 'str'
html_source[:500]

'<!DOCTYPE html>\n<html lang="vi">\n<head><meta http-equiv="content-type" content="text/html; charset=utf-8" /><base href="https://hosocongty.vn/"/>\n<meta name="robots" content="follow, all" />\n<meta name="keywords" content="Có 582,133 hồ sơ công ty tại Thành phố Hà Nội,Tra cứu mã số thuế, Tra cứu thông tin doanh nghiệp, Tra cứu doanh nghiêp, Tra cứu công ty, Hồ Sơ công ty"/>\n<meta name="description" content="Có 582,133 hồ sơ công ty tại Thành phố Hà Nội"/>\n<meta name="viewport" content="width=devi'

In [35]:
# The BeautifulSoup returns a document which we shall use to extract information
doc = BeautifulSoup(html_source, 'html.parser')

In [36]:
# validation: return datatype is bs4.BeautifulSoup
type(doc)

bs4.BeautifulSoup

In [37]:
#validation: inspect first 10 lines of the returned doc
doc.prettify().splitlines()[:10]

['<!DOCTYPE html>',
 '<html lang="vi">',
 ' <head>',
 '  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>',
 '  <base href="https://hosocongty.vn/"/>',
 '  <meta content="follow, all" name="robots"/>',
 '  <meta content="Có 582,133 hồ sơ công ty tại Thành phố Hà Nội,Tra cứu mã số thuế, Tra cứu thông tin doanh nghiệp, Tra cứu doanh nghiêp, Tra cứu công ty, Hồ Sơ công ty" name="keywords"/>',
 '  <meta content="Có 582,133 hồ sơ công ty tại Thành phố Hà Nội" name="description"/>',
 '  <meta content="width=device-width initial-scale=1.0" name="viewport"/>',
 '  <meta content="en_US" property="og:locale"/>']

In [64]:
#initialize variables
num = 25                #num of pages to crawl
urls = []               #list of company urls
infos = []              #list of company information
add = "https://hosocongty.vn/"


In [65]:
keyList = ["Name",
            "URL",
            "International name",
            "Abbreviations", 
            "Tax number", 
            "Address",
            "Legal representative",
            "Phone number","Email",
            "License date",
            "Business sector",
            "Status",
            "Last updated"]

for count in range(1, num+1):
    if count == 1:
        # download the web page using requests
        response = requests.get('https://hosocongty.vn/p/ho-chi-minh')
    else: 
        response = requests.get('https://hosocongty.vn/p/ho-chi-minh/page-'+str(count))

    # capture the text content of the page from response
    page_contents = response.text

    # Save the page as html to avoid scrapping multiple times while experimenting
    with open('ho_so_cong_ty.html', 'w', encoding="utf-8") as file:
        file.write(page_contents)

    # read it html page to scrape information
    with open('ho_so_cong_ty.html', 'r', encoding="utf-8") as f:
        html_source = f.read()

    # The BeautifulSoup returns a document which we shall use to extract information
    doc = BeautifulSoup(html_source, 'html.parser')
    for ul in doc.find_all('ul', class_ = 'hsdn'):
        # looking for anchor tag inside the <ul>tag
        for li in ul.find_all('li'):
            a = li.find('a')
            if a is None:
                continue
            else:
                print(a)
                print("--------")
                url = a.get('href')
                url = add + url
                # appending the url to the output list
                urls.append(url)
                page = requests.get(url)
                info = {key: None for key in keyList}
                info['URL'] = url
                soup = BeautifulSoup(page.content, 'html.parser')
                i = 0
                row = 0
                row1 = []
                row2 = []
                table = soup.find_all('ul', class_='hsct')
                while(row < 2):
                    #row1 = table[count].find_all('label')
                    #row2 = table[count].find_all('span')
                    text1 = table[row].find_all('label')          
                    text2 = table[row].find_all('span')
                    row1.extend(text1)
                    row2.extend(text2)
                    row = row + 1
                print("row1:")
                print(row1)
                print("row2:")
                print(row2)
                company_name = soup.find('h1')
                info['Name'] = company_name.text.strip()
                time = soup.find(lambda tag: tag.name == "li" and "Cập nhật lần cuối vào " in tag.text)
                if time != None:
                    timex = time.find_all('i')
                    info['Last updated'] = timex[1].text.strip()
                else:
                    info['Last updated'] = None
                size_row = len(row1)
                while (i < size_row): 
                    text = row1[i].text.strip()
                    if text == "☷ Tên quốc tế:":   
                        info['International name'] = row2[i].text.strip()
                    elif text == "☷ Tên viết tắt:":  
                        info['Abbreviations'] = row2[i].text.strip()
                    elif text == "Mã số thuế:":                      
                        info['Tax number'] = row2[i].text.strip()
                    elif text == "Địa chỉ thuế:":                      
                        info['Address'] = row2[i].text.strip()
                    elif text == "Đại diện pháp luật:":                      
                        info['Legal representative'] = row2[i].text.strip()
                    elif text == "Điện thoại:":                      
                        info['Phone number'] = row2[i].text.strip()
                    elif text == "Email:":                      
                        info['Email'] = row2[i].text.strip()
                    elif text == "Ngày cấp:":                      
                        info['License date'] = row2[i].text.strip()
                    elif text == "Ngành nghề chính:":                      
                        info['Business sector'] = row2[i].text.strip()
                    elif text == "Trạng thái:":                      
                        info['Status'] = row2[i].text.strip()
                    i = i + 1                
                infos.append(info)


<a href="cong-ty-tnhh-van-tuyen-ap-com-3033783.htm" title="0318007293 - CÔNG TY TNHH VÂN TUYỀN AP">CÔNG TY TNHH VÂN TUYỀN AP</a>
--------
row1:
[<label><i class="fa fa-hashtag"></i> Mã số thuế:</label>, <label><i class="fa fa-map-marker"></i> Địa chỉ thuế:</label>, <label><i class="fa fa-user-o"></i> Đại diện pháp luật:</label>, <label><i class="fa fa-calendar"></i> Ngày cấp:</label>, <label><i class="fa fa-anchor"></i> Ngành nghề chính:</label>, <label><i class="fa fa-info"></i> Trạng thái:</label>]
row2:
[<span>0318007293</span>, <span>137/39/4A Trịnh Đình Trọng, Phường Phú Trung, Quận Tân Phú, Hồ Chí Minh</span>, <span><a href="search?key=ĐẠI DIỆU VÂN&amp;opt=1" title="ĐẠI DIỆU VÂN">ĐẠI DIỆU VÂN</a></span>, <span> <a href="ngay-23/08/2023" title="Danh sách công ty thành lập ngày 23/08/2023">23/08/2023</a></span>, <span> <a href="industry-4663/ban-buon-vat-lieu-thiet-bi-lap-dat-khac-trong-xay-dung" title="Bán buôn vật liệu, thiết bị lắp đặt khác trong xây dựng">Bán buôn vật liệu, thi

In [66]:
# print all the information stored in the infos list
for info in infos:
    print(info)

{'Name': 'CÔNG TY TNHH VÂN TUYỀN AP', 'URL': 'https://hosocongty.vn/cong-ty-tnhh-van-tuyen-ap-com-3033783.htm', 'International name': None, 'Abbreviations': None, 'Tax number': '0318007293', 'Address': '137/39/4A Trịnh Đình Trọng, Phường Phú Trung, Quận Tân Phú, Hồ Chí Minh', 'Legal representative': 'ĐẠI DIỆU VÂN', 'Phone number': None, 'Email': None, 'License date': '23/08/2023', 'Business sector': 'Bán buôn vật liệu, thiết bị lắp đặt khác trong xây dựng', 'Status': 'Đang hoạt động', 'Last updated': '23/08/2023'}
{'Name': 'CÔNG TY TNHH TRUYỀN THÔNG SỞ HỮU TRÍ TUỆ VIỆT NAM', 'URL': 'https://hosocongty.vn/cong-ty-tnhh-truyen-thong-so-huu-tri-tue-viet-nam-com-3033775.htm', 'International name': None, 'Abbreviations': None, 'Tax number': '0318006388', 'Address': '6 Đặng Văn Sâm, Phường 2, Quận Tân Bình, TP Hồ Chí Minh', 'Legal representative': 'Nguyễn  Văn Diệu', 'Phone number': None, 'Email': None, 'License date': '22/08/2023', 'Business sector': 'Hoạt động dịch vụ hỗ trợ trực tiếp cho v

In [67]:
# create a dataframe
infos_df = pd.DataFrame(infos)
infos_df

,Name,URL,International name,Abbreviations,Tax number,Address,Legal representative,Phone number,Email,License date,Business sector,Status,Last updated
0,CÔNG TY TNHH VÂN TUYỀN AP,https://hosocongty.vn/cong-ty-tnhh-van-tuyen-a...,None,None,0318007293,"137/39/4A Trịnh Đình Trọng, Phường Phú Trung, ...",ĐẠI DIỆU VÂN,None,None,23/08/2023,"Bán buôn vật liệu, thiết bị lắp đặt khác trong...",Đang hoạt động,23/08/2023
1,CÔNG TY TNHH TRUYỀN THÔNG SỞ HỮU TRÍ TUỆ VIỆT NAM,https://hosocongty.vn/cong-ty-tnhh-truyen-thon...,None,None,0318006388,"6 Đặng Văn Sâm, Phường 2, Quận Tân Bình, TP Hồ...",Nguyễn Văn Diệu,None,None,22/08/2023,Hoạt động dịch vụ hỗ trợ trực tiếp cho vận tải...,Đang hoạt động,22/08/2023
2,CÔNG TY TNHH THƯƠNG MẠI DỊCH VỤ HỮU MINH TIẾN,https://hosocongty.vn/cong-ty-tnhh-thuong-mai-...,HUU MINH TIEN TRADE SERVICE COMPANY LIMITED,None,0318006980,"1926 Võ Văn Kiệt, Phường An Lạc, Quận Bình Tân...",PHẠM ĐĂNG KHOA,None,None,23/08/2023,"Sản xuất vali, túi xách và các loại tương tự, ...",Đang hoạt động,23/08/2023
3,CÔNG TY TNHH THƯƠNG MẠI DỊCH VỤ VẬN TẢI THẢO NGỌC,https://hosocongty.vn/cong-ty-tnhh-thuong-mai-...,None,None,0318006941,"Số 39 Đường số 10, Khu phố 2, Phường Phú Hữu, ...",NGUYỄN NGỌC THẢO,None,None,23/08/2023,Vận tải hàng hóa bằng đường bộ,Đang hoạt động,23/08/2023
4,CÔNG TY TNHH THƯƠNG MẠI DỊCH VỤ NHẬT THÔNG,https://hosocongty.vn/cong-ty-tnhh-thuong-mai-...,None,None,0318007141,"3/14 Võ Văn Tần, Phường Võ Thị Sáu, Quận 3, Hồ...",ĐẶNG VĂN PHÓNG,None,None,23/08/2023,Hoạt động kiến trúc và tư vấn kỹ thuật có liên...,Đang hoạt động,23/08/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,CÔNG TY TNHH GIẢI PHÁP TRUYỀN THÔNG TÍCH HỢP L...,https://hosocongty.vn/cong-ty-tnhh-giai-phap-t...,LEANLAB INTEGRATION MEDIA SOLUTIONS COMPANY LI...,"LEANLAB IMS CO., LTD",0318005923,"142/5 Lý Chính Thắng, Phường Võ Thị Sáu, Quận ...",NGUYỄN HỮU HIỀN,02838227687,None,23/08/2023,Quảng cáo,Đang hoạt động,23/08/2023
296,CÔNG TY TNHH SUNNYLEE MEDIA,https://hosocongty.vn/cong-ty-tnhh-sunnylee-me...,SUNNYLEE MEDIA COMPANY LIMITED,"SUNNYLEE MEDIA CO., LTD",0318005930,"số 6 Phùng Khắc Khoan, Phường Đa Kao, Quận 1, ...",Lê Châu Linh Phi,02838227687,None,23/08/2023,Quảng cáo,Đang hoạt động,23/08/2023
297,CÔNG TY TNHH DÒNG SÔNG XANH VIETNAM,https://hosocongty.vn/cong-ty-tnhh-dong-song-x...,GREEN RIVER VIETNAM COMPANY LIMITED,"GREEN RIVER VIETNAM CO., LTD",0318006130,"69 Nguyễn Thị Nhỏ, Phường 9, Quận Tân Bình, Hồ...",PHẠM THANH THU THỦY,,None,23/08/2023,Hoạt động dịch vụ hỗ trợ khác liên quan đến vậ...,Đang hoạt động,23/08/2023
298,CÔNG TY TNHH REAL CONSTRUCTION VIỆT NAM,https://hosocongty.vn/cong-ty-tnhh-real-constr...,"REAL CONSTRUCTION VIET NAM CO., LTD.",R.C.V,0318006148,"491/1 Trường Chinh, Phường 14, Quận Tân Bình, ...",NGUYỄN THANH THẢO,0902497860,None,23/08/2023,Hoạt động kiến trúc và tư vấn kỹ thuật có liên...,Đang hoạt động,23/08/2023


In [59]:
# convert dataframe to a csv file
infos_df.to_csv('companyTest.csv',index=None)